In [196]:
from catboost import CatBoostRegressor, Pool

import pandas as pd
from sklearn.model_selection import KFold
from scipy.stats import spearmanr
from sklearn.metrics import mean_squared_error

df = pd.read_csv('X_imputed_DE.csv').drop(['ID',"DAY_ID","DE_RAIN","DE_WIND","DE_TEMP"], axis=1)
labels=pd.read_csv('Y_imputed_DE.csv').drop('ID', axis=1)
labels["Rank_2"]=labels["Rank"]**2

# df_FR = pd.read_csv('X_imputed_FR.csv').drop(['ID',"DAY_ID"], axis=1)
df_FR = pd.read_csv('X_imputed_FR.csv').drop(['ID',"DAY_ID","FR_DE_EXCHANGE","FR_NET_EXPORT","FR_RAIN","FR_WIND","FR_TEMP"], axis=1)
labels_FR=pd.read_csv('Y_imputed_FR.csv').drop('ID', axis=1)

list(df.columns)

['DE_CONSUMPTION',
 'DE_FR_EXCHANGE',
 'DE_NET_EXPORT',
 'DE_GAS',
 'DE_COAL',
 'DE_HYDRO',
 'DE_NUCLEAR',
 'DE_SOLAR',
 'DE_WINDPOW',
 'DE_LIGNITE',
 'DE_RESIDUAL_LOAD',
 'GAS_RET',
 'COAL_RET',
 'CARBON_RET']

In [199]:

from Brute_force_feature_engineering import *

base_columns=list(df.columns)
df_copy,corr=main_brute(df,base_columns,labels,n_uplet=20,thresh=0.25)
print(corr)
df_copy2=df_copy[list(corr.index)+base_columns]

powers added
16369 colunms to add


  0%|          | 0/16369 [00:00<?, ?it/s]

100%|██████████| 16369/16369 [01:02<00:00, 262.57it/s]


n_uplet added
_x_CARBON_RET_x_COAL_RET_x_DE_LIGNITE_x_DE_WINDPOW_x_DE_HYDRO_x_DE_GAS_x_DE_NET_EXPORT_x_DE_CONSUMPTION                                                                       0.265474
_x_CARBON_RET_x_COAL_RET_x_DE_LIGNITE_x_DE_WINDPOW_x_DE_SOLAR_x_DE_HYDRO_x_DE_COAL_x_DE_GAS_x_DE_CONSUMPTION                                                                  0.264804
_x_CARBON_RET_x_COAL_RET_x_DE_LIGNITE_x_DE_WINDPOW_x_DE_SOLAR_x_DE_HYDRO_x_DE_GAS_x_DE_FR_EXCHANGE_x_DE_CONSUMPTION                                                           0.261189
_x_CARBON_RET_x_COAL_RET_x_GAS_RET_x_DE_LIGNITE_x_DE_SOLAR_x_DE_COAL_x_DE_NET_EXPORT_x_DE_CONSUMPTION                                                                         0.258630
_x_CARBON_RET_x_COAL_RET_x_GAS_RET_x_DE_LIGNITE_x_DE_WINDPOW_x_DE_COAL_x_DE_GAS_x_DE_NET_EXPORT_x_DE_FR_EXCHANGE_x_DE_CONSUMPTION                                             0.258111
_x_CARBON_RET_x_GAS_RET_x_DE_LIGNITE_x_DE_WINDPOW_x_DE_SOLAR_x_DE_HYDRO

In [198]:
base_columns_FR=list(df_FR.columns)
df_copy_FR,corr_FR=main_brute(df_FR,base_columns_FR,labels_FR,n_uplet=9,thresh=0.13)
print(corr_FR)
df_copy2_FR=df_copy_FR[list(corr_FR.index)+base_columns_FR]

powers added
2024 colunms to add


100%|██████████| 2024/2024 [00:03<00:00, 529.12it/s]


n_uplet added
_x_GAS_RET_x_FR_WINDPOW                                                                           0.153600
_x_CARBON_RET_x_GAS_RET                                                                           0.150299
_x_CARBON_RET_x_GAS_RET_x_FR_RESIDUAL_LOAD_x_FR_GAS                                               0.142039
_x_CARBON_RET_x_GAS_RET_x_FR_SOLAR_x_FR_CONSUMPTION                                              -0.132993
_x_CARBON_RET_x_GAS_RET_x_FR_SOLAR_x_FR_NUCLEAR_x_FR_HYDRO_x_FR_GAS                              -0.133298
_x_CARBON_RET_x_GAS_RET_x_FR_RESIDUAL_LOAD_x_FR_SOLAR_x_FR_NUCLEAR_x_FR_HYDRO_x_FR_CONSUMPTION   -0.133704
_x_CARBON_RET_x_GAS_RET_x_FR_RESIDUAL_LOAD_x_FR_SOLAR_x_FR_HYDRO_x_FR_GAS_x_FR_CONSUMPTION       -0.133785
_x_CARBON_RET_x_GAS_RET_x_FR_SOLAR_x_FR_HYDRO_x_FR_GAS                                           -0.135605
_x_CARBON_RET_x_GAS_RET_x_FR_SOLAR_x_FR_NUCLEAR_x_FR_GAS_x_FR_CONSUMPTION                        -0.136024
_x_CARBON_RET_x_GAS_RET

In [200]:
inf_mask = df_copy.isin([np.inf])
df_copy.loc[inf_mask.any(axis=1), inf_mask.any(axis=0)]

""


In [201]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train_clean_DE = df
Y_train_clean_DE = labels
X_train_clean_FR = df_FR
Y_train_clean_FR = labels_FR
X_train_clean_FR, X_test_FR, Y_train_clean_FR, y_test_FR = train_test_split(X_train_clean_FR, Y_train_clean_FR, test_size=0.25)
X_train_clean_DE, X_test_DE, Y_train_clean_DE, y_test_DE = train_test_split(X_train_clean_DE, Y_train_clean_DE, test_size=0.25)

# print (output_train[:,1])
def metric_train(output,df):
    term=output[0]
    if not isinstance(term, np.ndarray):
        return  100 *spearmanr(output, df["TARGET"]).correlation
    else:
        return  [100 *spearmanr(output[:,i], df["TARGET"]).correlation for i in range(len(term)) ]

In [202]:
def average_stats(df,labels,regressor,columns=['TARGET','Rank'],iterations=20,depth=None,group=False):
    valid_list=[]
    for i in range(iterations):
        X_train_clean_DE, X_test_DE, Y_train_clean_DE, y_test_DE = train_test_split(df, labels, test_size=0.25)
        if group: 
            Y_train_clean_DE['Rank_group']=pd.qcut(Y_train_clean_DE.Rank,30).cat.codes
            if columns is None:
                columns= 'Rank_group'
            elif 'Rank_group'  not in columns: 
                if isinstance(columns,list):
                    columns=columns+['Rank_group']
                else:
                    columns=[columns]+['Rank_group']
        if depth is None:
            DF = regressor()
        else:            
            DF = regressor(max_depth=depth)
        #   Fit the ridge regressor

        DF.fit(X_train_clean_DE, Y_train_clean_DE[columns])
        output_train = DF.predict(X_test_DE)
        # print(output_train)
        validation=metric_train(output_train,y_test_DE)
        
        valid_list+=[validation]
    
    if isinstance(columns,list):
        for i,column in enumerate(columns):
            print( f"average {column} score for ", sum([row[i] for row in valid_list])/len(valid_list) )
    else:
        print( f"average {columns} score for ", sum(valid_list)/len(valid_list) )
    
    print("\n")


In [203]:
def average_stats_(df,labels,regressor,columns=['TARGET','Rank'],iterations=20,depth=None,group=False):
    valid_list=[]
    for i in range(iterations):
        X_train_clean_DE, X_test_DE, Y_train_clean_DE, y_test_DE = train_test_split(df, labels, test_size=0.25)
        if group: 
            Y_train_clean_DE['Rank_group']=pd.qcut(Y_train_clean_DE.Rank,30).cat.codes
            if columns is None:
                columns= 'Rank_group'
            elif 'Rank_group'  not in columns: 
                if isinstance(columns,list):
                    columns=columns+['Rank_group']
                else:
                    columns=[columns]+['Rank_group']
        if depth is None:
            DF = regressor()
        else:            
            DF = regressor(max_depth=depth)
        #   Fit the ridge regressor

        DF.fit(X_train_clean_DE, Y_train_clean_DE[columns])
        output_train = DF.predict(X_test_DE)
        # print(output_train)
        validation=metric_train(output_train,y_test_DE)
        
        valid_list+=[validation]
    
    if isinstance(columns,list):
        for i,column in enumerate(columns):
            print( f"average {column} score for ", sum([row[i] for row in valid_list])/len(valid_list) )
    else:
        print( f"average {columns} score for ", sum(valid_list)/len(valid_list) )
    
    print("\n")


In [204]:
from sklearn.ensemble import RandomForestRegressor

average_stats(df_copy2,labels,RandomForestRegressor,depth=3)
average_stats(df_copy2,labels,RandomForestRegressor,columns='Rank',depth=3)
average_stats(df_copy2,labels,RandomForestRegressor,columns='TARGET',depth=3)

average TARGET score for  32.352674136333746
average Rank score for  32.84419846055667


average Rank score for  34.93084012878229


average TARGET score for  26.659680594798015




In [205]:
average_stats(df,labels,RandomForestRegressor,depth=3)
average_stats(df,labels,RandomForestRegressor,columns='Rank',depth=3)
average_stats(df,labels,RandomForestRegressor,columns='TARGET',depth=3)


average TARGET score for  32.75692439453148
average Rank score for  33.262689952599224


average Rank score for  34.76429066110044


average TARGET score for  25.265252729714547




In [206]:
average_stats(df,labels,RandomForestRegressor,columns=None,depth=3,group=True)
average_stats(df_copy2,labels,RandomForestRegressor,columns=None,depth=3,group=True)

average Rank_group score for  36.30638250694482


average Rank_group score for  30.68978835793998




In [207]:
average_stats(df_FR,labels_FR,RandomForestRegressor,depth=3)
average_stats(df_FR,labels_FR,RandomForestRegressor,columns='Rank',depth=3)
average_stats(df_FR,labels_FR,RandomForestRegressor,columns='TARGET',depth=3)

average TARGET score for  17.524418953781517
average Rank score for  19.002513666718052


average Rank score for  18.585382918594313


average TARGET score for  8.441262153374547




In [208]:
average_stats(df_copy2_FR,labels_FR,RandomForestRegressor,depth=3)
average_stats(df_copy2_FR,labels_FR,RandomForestRegressor,columns='Rank',depth=3)
average_stats(df_copy2_FR,labels_FR,RandomForestRegressor,columns='TARGET',depth=3)

average TARGET score for  19.429634761550854
average Rank score for  20.560434314123267


average Rank score for  20.69651055626914


average TARGET score for  14.259310752491492




In [209]:
average_stats(df_copy2_FR,labels_FR,RandomForestRegressor,columns=None,depth=3,group=True)
average_stats(df_FR,labels_FR,RandomForestRegressor,columns=None,depth=3,group=True)

average Rank_group score for  18.46192922678766


average Rank_group score for  17.67332174097308




In [210]:
from mord import OrdinalRidge,LAD
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    average_stats(df_copy2,labels,OrdinalRidge,iterations=100)
    average_stats(df_copy2,labels,OrdinalRidge,columns='Rank',iterations=100)
    average_stats(df_copy2,labels,OrdinalRidge,columns='TARGET',iterations=100)
    average_stats(df_copy2,labels,OrdinalRidge,columns=None, group=True,iterations=100)
    
    
    average_stats(df,labels,OrdinalRidge,iterations=100)
    average_stats(df,labels,OrdinalRidge,columns='Rank',iterations=100)
    average_stats(df,labels,OrdinalRidge,columns='TARGET',iterations=100)
    average_stats(df,labels,OrdinalRidge,columns=None, group=True,iterations=100)
    

average TARGET score for  10.874659238669448
average Rank score for  30.935364068064494


average Rank score for  29.5422527335191


average TARGET score for  11.008569212319385


average Rank_group score for  29.886952490934064


average TARGET score for  19.432688024490634
average Rank score for  37.99639649742855


average Rank score for  37.43823405406133


average TARGET score for  18.25836124093441


average Rank_group score for  37.61988403737143




In [211]:
from mord import OrdinalRidge,LAD
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    average_stats(df_copy2_FR,labels_FR,OrdinalRidge,iterations=100)
    average_stats(df_copy2_FR,labels_FR,OrdinalRidge,columns='Rank',iterations=100)
    average_stats(df_copy2_FR,labels_FR,OrdinalRidge,columns='TARGET',iterations=100)
    average_stats(df_copy2_FR,labels_FR,OrdinalRidge,columns=None, group=True,iterations=100)
    
    
    average_stats(df_FR,labels_FR,OrdinalRidge,iterations=100)
    average_stats(df_FR,labels_FR,OrdinalRidge,columns='Rank',iterations=100)
    average_stats(df_FR,labels_FR,OrdinalRidge,columns='TARGET',iterations=100)
    average_stats(df_FR,labels_FR,OrdinalRidge,columns=None, group=True,iterations=100)
    

average TARGET score for  nan
average Rank score for  16.681366495657585


average Rank score for  15.245328766098751


average TARGET score for  3.483115463774554


average Rank_group score for  16.927773299304082


average TARGET score for  nan
average Rank score for  18.802887219049854


average Rank score for  19.980412204254016


average TARGET score for  nan


average Rank_group score for  17.812501983223473




In [10]:
from mord import OrdinalRidge,LAD
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    average_stats_ord(df_copy2,labels,OrdinalRidge,100)
    average_stats_ord(df,labels,OrdinalRidge,100)

    average_stats_ord(df_copy2_FR,labels_FR,OrdinalRidge,20)
    average_stats_ord(df_FR,labels_FR,OrdinalRidge,100)
# average_stats_ord(df_copy2,labels,LAD,20)
# average_stats_ord(df,labels,LAD,20)

average target score for  37.3517693287154


average target score for  35.0827280723113


average target score for  23.198594512815276


average target score for  20.744098194867448




In [175]:
Y_train_clean_DE['Rank_group']=pd.qcut(Y_train_clean_DE.Rank,31).cat.codes

Y_train_clean_FR['Rank_group']=pd.qcut(Y_train_clean_FR.Rank,10).cat.codes

In [176]:
Y_train_clean_DE['Rank_group']

583     3
267     9
333    19
92      3
173    30
       ..
324    26
221    18
132     1
165     6
355    28
Name: Rank_group, Length: 447, dtype: int8

In [181]:
import lightgbm as lgb
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'boosting_type': 'gbdt',
    'learning_rate': 0.005,
    'num_leaves': 30,
    'n_estimators': 100
}

DF_DE = lgb.LGBMRanker(**params)
DF_DE.fit(X_train_clean_DE, Y_train_clean_DE['Rank_group'],group=[len(X_train_clean_DE)])
output_train = DF_DE.predict(X_test_DE)
# print(output_train)
print('Spearman correlation for the validation set', ( metric_train(output_train,y_test_DE)))
print('Spearman correlation for the train set', ( metric_train(DF_DE.predict(X_train_clean_DE),Y_train_clean_DE)))

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.105297 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2089
[LightGBM] [Info] Number of data points in the train set: 447, number of used features: 14
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spl

In [142]:
Y_train_clean_DE['Rank'].astype(int)

583     143
267     386
333    1019
92      162
173    1463
       ... 
324    1329
221    1007
132      71
165     255
355    1393
Name: Rank, Length: 447, dtype: int64

In [148]:
from sklearn.ensemble import RandomForestClassifier
DF_DE = RandomForestClassifier(max_depth=10)
# Fit the ridge regressor
DF_DE.fit(X_train_clean_DE, Y_train_clean_DE['Rank'].astype(int))
output_train = DF_DE.predict(X_test_DE)
# print(output_train)
print('Spearman correlation for the validation set', ( metric_train(output_train,y_test_DE)))
print('Spearman correlation for the train set', ( metric_train(DF_DE.predict(X_train_clean_DE),Y_train_clean_DE)))

Spearman correlation for the validation set 4.981097333797748
Spearman correlation for the train set 86.03621734681876


In [136]:
from sklearn.ensemble import RandomForestRegressor
DF_DE = RandomForestRegressor(max_depth=2)
# Fit the ridge regressor
DF_DE.fit(X_train_clean_DE, Y_train_clean_DE['Rank_group'])
output_train = DF_DE.predict(X_test_DE)
# print(output_train)
print('Spearman correlation for the validation set', ( metric_train(output_train,y_test_DE)))
print('Spearman correlation for the train set', ( metric_train(DF_DE.predict(X_train_clean_DE),Y_train_clean_DE)))

Spearman correlation for the validation set 37.782435468402504
Spearman correlation for the train set 51.70538304981839


In [28]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
XGB_DE = XGBRegressor(max_depth=1)
# Fit the ridge regressor
XGB_DE.fit(X_train_clean_DE, Y_train_clean_DE)
output_train = XGB_DE.predict(X_test_DE)
# print(output_train)
print('Spearman correlation for the Validation set', ( metric_train(output_train,y_test_DE)))
print('Spearman correlation for the train set', ( metric_train(XGB_DE.predict(X_train_clean_DE),Y_train_clean_DE)))

Spearman correlation for the Validation set [18.11907364009942, 24.937080479858324]
Spearman correlation for the train set [59.8641556609763, 66.96166829593699]


In [165]:
from sklearn.ensemble import RandomForestClassifier
DF_FR = RandomForestClassifier(max_depth=6)
# Fit the ridge regressor
DF_FR.fit(X_train_clean_FR, Y_train_clean_FR['Rank'].astype(int))
output_train = DF_FR.predict(X_test_FR)
# print(output_train)
print('Spearman correlation for the Validation set', ( metric_train(output_train,y_test_FR)))
print('Spearman correlation for the train set', ( metric_train(DF_FR.predict(X_train_clean_FR),Y_train_clean_FR)))

Spearman correlation for the Validation set 14.493500998036932
Spearman correlation for the train set 64.96589995260416


In [66]:
from sklearn.ensemble import RandomForestRegressor
DF_FR = RandomForestRegressor(max_depth=6)
# Fit the ridge regressor
DF_FR.fit(X_train_clean_FR, Y_train_clean_FR['Rank_group'])
output_train = DF_FR.predict(X_test_FR)
# print(output_train)
print('Spearman correlation for the Validation set', ( metric_train(output_train,y_test_FR)))
print('Spearman correlation for the train set', ( metric_train(DF_FR.predict(X_train_clean_FR),Y_train_clean_FR)))

Spearman correlation for the Validation set 20.38084413708077
Spearman correlation for the train set 82.4383230479599


In [41]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
XGB_FR = XGBRegressor(max_depth=1)
# Fit the ridge regressor
XGB_FR.fit(X_train_clean_FR, Y_train_clean_FR)
output_train = XGB_FR.predict(X_test_FR)
# print(output_train)
print('Spearman correlation for the Validation set', ( metric_train(output_train,y_test_FR)))
print('Spearman correlation for the train set', ( metric_train(XGB_FR.predict(X_train_clean_FR),Y_train_clean_FR)))

Spearman correlation for the Validation set [13.437535531717446, 17.771587690073098]
Spearman correlation for the train set [41.232491128221305, 55.59977310447102]


In [202]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
LGB_FR = LGBMRegressor(max_depth=1)
# Fit the ridge regressor
LGB_FR.fit(X_train_clean_FR, Y_train_clean_FR["Rank"])
output_train = LGB_FR.predict(X_test_FR)
print(output_train[0])
print('Spearman correlation for the Validation set', ( metric_train(output_train,y_test_FR)))
print('Spearman correlation for the train set', ( metric_train(LGB_FR.predict(X_train_clean_FR),Y_train_clean_FR)))

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000554 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2018
[LightGBM] [Info] Number of data points in the train set: 550, number of used features: 11
[LightGBM] [Info] Start training from score 728.219091
693.1255833929005
Spearman correlation for the Validation set 21.685499480320768
Spearman correlation for the train set 38.60464097289937


In [220]:
X_test=pd.read_csv("X_test_final.csv")
data=X_test.drop('COUNTRY',axis=1)
# Fill missing values with the mean (you can choose other imputation strategies as well)
X_test.fillna(data.mean(), inplace=True)
inf_mask = X_test.isna()
X_test.loc[inf_mask.any(axis=1), inf_mask.any(axis=0)].columns

Index([], dtype='object')

In [84]:

X_test=X_test.drop(["FR_NET_IMPORT","DE_NET_IMPORT","FR_DE_EXCHANGE","FR_NET_EXPORT","DE_WIND","FR_WIND","DE_RAIN","FR_RAIN","DE_TEMP","FR_TEMP"],axis=1)
X_test.head()

# Create sub-dataset 'trainX_DE' with rows where ID is 'DE' and keep columns starting with 'ID'
X_test_DE = X_test[X_test['COUNTRY'].str.startswith('DE')].loc[:, X_test.columns.str.contains('^(ID|DE|GAS|COAL|CARBON)')]


# Create sub-dataset 'trainX_FR' with rows where ID is 'FR' and keep columns starting with 'ID'
X_test_FR = X_test[X_test['COUNTRY'].str.startswith('FR')].loc[:, X_test.columns.str.contains('^(ID|FR|GAS|COAL|CARBON)')]

/tmp/ipykernel_1522238/2923754365.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  X_test_DE = X_test[X_test['COUNTRY'].str.startswith('DE')].loc[:, X_test.columns.str.contains('^(ID|DE|GAS|COAL|CARBON)')]
/tmp/ipykernel_1522238/2923754365.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  X_test_FR = X_test[X_test['COUNTRY'].str.startswith('FR')].loc[:, X_test.columns.str.contains('^(ID|FR|GAS|COAL|CARBON)')]


In [85]:
print(X_test_FR.isnull().sum())
print(X_test_DE.isnull().sum())


ID                  0
FR_CONSUMPTION      0
FR_GAS              0
FR_COAL             0
FR_HYDRO            0
FR_NUCLEAR          0
FR_SOLAR            0
FR_WINDPOW          0
FR_RESIDUAL_LOAD    0
GAS_RET             0
COAL_RET            0
CARBON_RET          0
dtype: int64
ID                  0
DE_CONSUMPTION      0
DE_FR_EXCHANGE      0
DE_NET_EXPORT       0
DE_GAS              0
DE_COAL             0
DE_HYDRO            0
DE_NUCLEAR          0
DE_SOLAR            0
DE_WINDPOW          0
DE_LIGNITE          0
DE_RESIDUAL_LOAD    0
GAS_RET             0
COAL_RET            0
CARBON_RET          0
dtype: int64


In [110]:
X_pred_FR=DF_FR.predict(X_test_FR.drop("ID",axis=1))
X_pred_DE=DF_DE.predict(X_test_DE.drop("ID",axis=1))


array([[-1.87386115e-01,  5.38745000e+02],
       [ 6.88223855e-01,  7.69880000e+02],
       [-8.14621016e-01,  3.20205000e+02],
       [ 5.23744779e-01,  9.58680000e+02],
       [ 7.17551427e-02,  7.72320000e+02],
       [ 4.04714404e-02,  7.29975000e+02],
       [ 3.04658517e-01,  8.07270000e+02],
       [ 3.76156778e-01,  7.51595000e+02],
       [ 4.54008943e-02,  5.93965000e+02],
       [-8.41711968e-01,  2.40195000e+02],
       [ 3.40854929e-01,  8.13930000e+02],
       [ 4.09560705e-01,  8.85690000e+02],
       [ 9.25010231e-01,  1.08611000e+03],
       [ 4.24942478e-01,  9.15030000e+02],
       [ 4.35795427e-02,  7.22660000e+02],
       [ 6.86836110e-02,  6.85680000e+02],
       [ 1.28556128e-01,  7.28510000e+02],
       [ 6.03782929e-01,  1.00260000e+03],
       [ 2.55187580e-01,  8.31810000e+02],
       [ 1.71038854e-01,  7.92750000e+02],
       [ 2.16358019e-01,  7.70050000e+02],
       [ 2.33589427e-01,  7.27685000e+02],
       [-1.03219461e+00,  2.03595000e+02],
       [ 5.

In [118]:
RANK_FR = pd.DataFrame({'ID': X_test_FR["ID"], 'TARGET': X_pred_FR[:,1]})
TARGET_FR = pd.DataFrame({'ID': X_test_FR["ID"], 'TARGET': X_pred_FR[:,0]})
RANK_DE = pd.DataFrame({'ID': X_test_DE["ID"], 'TARGET': X_pred_DE[:,1]})
TARGET_DE = pd.DataFrame({'ID': X_test_DE["ID"], 'TARGET': X_pred_DE[:,0]})

In [119]:
TARGET_pred=pd.concat([TARGET_DE, TARGET_FR]).sort_index()
RANK_pred=pd.concat([RANK_DE, RANK_FR]).sort_index()

In [120]:
RANK_pred.to_csv('RANK_pred.csv', index=False)
TARGET_pred.to_csv('TARGET_pred.csv', index=False)

In [121]:
pd.read_csv('RANK_pred.csv')

,ID,TARGET
0,1115,689.335
1,1202,707.100
2,1194,478.540
3,1084,792.150
4,1135,717.485
...,...,...
649,879,222.610
650,673,687.200
651,1641,987.610
652,712,968.410
